# Paillier Crypto

## Generate Public and Private Key

In [1]:
from openfed.hooks.cypher.paillier_crypto import key_gen
import os
import torch

if not os.path.isfile('/tmp/public.key') or not os.path.isfile('/tmp/private.key'):
    public_key, private_key = key_gen()
    torch.save(public_key, '/tmp/public.key')
    torch.save(private_key, '/tmp/private.key')
    print("Save new key to /tmp/public.key and /tmp/private.key")
else:
    private_key = torch.load('/tmp/private.key')
    print("Load private key from /tmp/private.key")
    print(private_key)

Load private key from /tmp/private.key
<OpenFed> PrivateKey
n_lwe: 3000, bits: 32, bits_safe: 24, l: 64, bound: 8, p: 4294967297, q: 4294967296



## Network

In [2]:
import torch.nn as nn

network = nn.Linear(784, 10)
loss_fn = nn.CrossEntropyLoss()

## Aggregator

In [3]:
from openfed.optim import PaillierOp, build_aggregator

agg_op = PaillierOp(network.parameters(), private_key)

aggregator = build_aggregator(agg_op)

print(aggregator)

<OpenFed> Gluer
Gluer_PaillierOp_ReduceOp



## Build Optimizer

In [4]:
import torch

from openfed.optim import build_fed_optim

optim = torch.optim.SGD(network.parameters(), lr=1.0)
fed_optim = build_fed_optim(optim)

print(fed_optim)

<OpenFed> Gluer
Gluer_SGD_Penalizer



## Topology

In [5]:
import openfed

server_node = openfed.topo.Node('server', openfed.default_tcp_address, mtt=5)
client = openfed.topo.Node('client', openfed.empty_address, mtt=5)

topology = openfed.topo.Topology()
topology.add_edge(client, server_node)

federated_group_props = topology.topology_analysis(server_node)[0]

print(federated_group_props)

openfed_leader, server, mtt=5
Address(backend='gloo', init_method='tcp://localhost:1994', world_size=2, rank=0)


## API

In [6]:
from openfed import API
openfed_api = API(
    state_dict=network.state_dict(keep_vars=True),
    fed_optim=fed_optim,
    aggregator=aggregator)

print(openfed_api)

# Step

In [7]:
from openfed.hooks import Aggregate

with openfed_api: 
    aggregate = Aggregate(
        activated_parts=dict(train=2),
        max_version=50,
    )
    print(aggregate)

<OpenFed> AggregateStep
period: 1 day, 0:00:00, activated_parts: {'train': 2}, checkpoint: None
max_loop_times: -1, max_version: 50



## Connection

In [8]:
openfed_api.build_connection(federated_group_props)

## Loop

In [9]:
for r in range(5):
    openfed_api.step()

<Round: 5> train: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]


## Finished

In [10]:
openfed_api.finish()

print("Finished")

Finished
